# Сборный проект №1

## Откроем файл с данными и изучем общую информацию

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import bar_chart_race as bcr

In [ ]:
data = pd.read_csv('/workspaces/Data-Science/Dataset/games.csv')
data.head()

* основную информацию для датафрейма `data` с помощью метода `info()`

## Подготовим данные
### Заменим названия столбцов (приведем к нижнему регистру)

In [ ]:
data.columns = data.columns.str.lower()
data.info()

In [ ]:
data.isna().sum()# подсчёт пропусков

* Данные в колонках 'year_of_release' можно преоброзовать в  формат datatime. 
* Данные в колонках 'critic_score' 'user_score' можно преоброзовать в  числовой тип. 
* Пропуски по годам незначительные. 
* Пропуски критиков и пользователей заменим на заглушки. 
* 'user_score' есть текст. Надо подробнее расмотреть
* Данные операции произведем ниже

<div class="alert alert-info"> <b>Комментарий студента:</b> Добавил подсчет пропусков, а также. Указал особенности данных </div>

### Преобразуем данные в нужные типы

In [ ]:
data['critic_score'].unique()# уникальные значения в колоке оценка критиков (максимум 100)

In [ ]:
data['critic_score'].isna().sum()#количество пропущенных значений оценка критиков

In [ ]:
data['critic_score'] = data['critic_score'].fillna(-1)# примем для отсутвующих значений оценки критиков "-1"
data['critic_score'] = data['critic_score'].astype(int)# преобразуем в тип int
data['critic_score'].unique()

In [ ]:
import datetime
data['year_of_release'].isna().sum()#количество пропущенных значений годе выпуска

In [ ]:
data['year_of_release'].unique()# уникальные значения в годе выпуска

In [ ]:
data = data.dropna(subset=['year_of_release']).reset_index(drop=True)# удалим игры с пропущенными признаками года выпуска
data.info()

In [ ]:
data['year_of_release'] = pd.to_datetime(data['year_of_release'], format='%Y')
#переводим столбец в формат datetime
data['year_of_release'] = data['year_of_release'].dt.year #вытягивает год из колонки

In [ ]:
data['user_score'].unique()# уникальные значения в колоке оценка пользователей (максимум 10)

In [ ]:
data[data['user_score'] == 'tbd'].groupby(
    ['year_of_release','platform','name']).agg(
    {'user_score':'count'}).sort_values(by='user_score', ascending=False).head(10)

* Получается что 'tbd' означает to be determined: недостаточно пользовательских оценок для формирования рейтинга - можно заменить на '-2', а nan заменить на '-1'. Чтоб отличать эти значения
* Рейтиг переведем в 100 бальную шкалу 

In [ ]:
data['user_score'] = data['user_score'].replace("tbd", -0.2) #присваиваем значению tbd = -0.2
data['user_score'] = data['user_score'].fillna(-0.1)#присваиваем значению nan = -0.1
data['user_score'] = data['user_score'].astype(float)#меняем тип на численные значения
data['user_score'] = data['user_score']*10#переводим десятиричный рейтинг в 100 бальный
data['user_score'] = data['user_score'].astype(int)#меняем тип на целые значения
data['user_score'].unique()# уникальные значения в колоке оценка пользователей (максимум 100)

In [ ]:
data['rating'].unique()

In [ ]:
data['rating'] = data['rating'].fillna('unknown')#присваиваем значению nan 'unknown'

In [ ]:
data[data['rating'] == 'unknown'].groupby(
    ['na_sales','eu_sales','other_sales', 'jp_sales']).agg(
    {'user_score':'count'}).sort_values(by='user_score', ascending=False).head(20)

* Entertainment Software Rating Board (ESRB) — негосударственная организация, основное направление деятельности — принятие и определение рейтингов для компьютерных видеоигр и другого развлекательного программного обеспечения в США и Канаде.
* Больше всего пропусков в рейтинге у игр которые продавались за пределами Севенй Америки, что логично.

In [ ]:
data[data['name'].isnull()]#эти данные можно удалить, они не представляют интереса

In [ ]:
data = data.dropna()#удаляем две строки без названия

In [ ]:
data.info()

In [ ]:
data['word_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']

In [ ]:
data['platform'].unique()

In [ ]:
data['genre'].unique()

In [ ]:
data.duplicated().sum()# подсчёт явных дубликатов

###  Обработали данные
* 1. Рейтинг критиков и пользователей привели к целым числам и сто бальной шкале
* 2. Пропуски в этих рейтингах заменили на особое значение '-1'
* 3. 'tbd' заменили на особое значение '-2'
* 3. Пропуске в годах заменили на '2022'
* 4. Пропуски в возрастной категории заменили на 'unknown'. Entertainment Software Rating Board (ESRB) — негосударственная организация, основное направление деятельности — принятие и определение рейтингов для компьютерных видеоигр и другого развлекательного программного обеспечения в США и Канаде. Больше всего пропусков в рейтинге у игр которые продавались за пределами Севенй Америки, что логично.
* 5. 'word_sales' колонка суммы продаж в мире в млн копий
* 6. Удалили оставшиеся пропуски в названиях
* 7. Проверили явные дубликаты

## Проведем исследовательский анализ данных

### Посмотрим, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?

In [ ]:
data_year = data.pivot_table(index="year_of_release", values=["name"], aggfunc="count")
plt.figure(figsize=(5, 5))
plt.plot(data_year, 'v-.g', alpha=0.4, label="сколько игр выпускалось в разные годы", 
         lw=2, mec='r', mew=2, ms=5)
plt.legend()
plt.grid(True)

* До 1995 года выпускалось слишком мало игр. Их можно отброосить.
* Пик продаж пришелся на 2005-2011. Больше всего было продано в 2008-2009 годах

In [ ]:
data = data[data['year_of_release'] > 1994]

Посмотрим, как менялись продажи по платформам. Выберим платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?

In [ ]:
data.pivot_table(index="year_of_release", columns="platform", 
                 values="word_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(20, 10), grid=True);


* Новые платформы повяются примерно каждые пять лет, старые изчезают примерно через один, два года после появления новых

Возьмем данные за соответствующий актуальный период. Основной фактор — эти данные помогут построить прогноз на 2017 год. С 2012 по 2016 (за 2016 данные не полные, новых платформ не появилось, но пропорция сохраняется)

In [ ]:
data_topical = data.query('year_of_release > 2011 and year_of_release < 2017')#актуальный период
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="word_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="jp_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="na_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="eu_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="other_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);

In [ ]:
data_topical.pivot_table(index="year_of_release", 
columns="platform", values="word_sales", aggfunc="sum")

* Лидируют по продажам PS4, XOne появилась в 2013 году и набирает популярность. У остальных продажи падают. Общая тенденция к сокращению продаж копии.

### Построим график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишим результат.

In [ ]:
colors = ['#78C850', '#F08030', '#6890F0','#F8D030', '#F85888', '#705898', '#98D8D8'] # цвета
sns.boxplot(x=data_topical["word_sales"], y=data_topical["platform"], 
            palette=colors, order=["PS4", "PS3", "XOne",  "X360", "3DS", "WiiU", 'Wii']);
plt.xlim(0,3) 
# указали цвета и очередность по количествам продаж

In [ ]:
colors = ['#78C850', '#F08030', '#6890F0','#F8D030', '#F85888', '#705898', '#98D8D8'] # цвета
sns.boxplot(x=data_topical["word_sales"], y=data_topical["platform"], 
            palette=colors, order=["PS4", "PS3", "XOne",  "X360", "3DS", "WiiU", 'Wii']);
# указали цвета и очередность по количествам продаж

* У PS3 и PS4 одинаковые медиальные значения. Продажи им сделали 4 - 6 игр. 
* X360 аналогичная ситуация, но среднее медиальное больше
* XOne менее всего выбросов. Более равномерное распределение и медеальное чуть меньше X360
* 3 DS несколько игр которые сделали продажи платформе. Притом довольно малое медиальное значение
* WiiU и Wii есть несколько игр, за счет чего выбивается из средних показателей

### Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами.

In [ ]:
data_XOne = data_topical[data_topical['platform'] == 'XOne'].groupby('user_score')['word_sales'].mean()
data_XOne2 = data_topical[data_topical['platform'] == 'XOne'].groupby('critic_score')['word_sales'].mean()
plt.figure(figsize=(5, 5))
plt.plot(data_XOne2, 'o-r', alpha=0.9, label="влияние продаж от оценки критиков XOne", 
         lw=5, mec='b', mew=1, ms=5)
plt.plot(data_XOne, 'v-.g', alpha=0.4, label="влияние продаж от оценки пользователя XOne", 
         lw=2, mec='r', mew=2, ms=5)
plt.legend()
plt.grid(True)

* Для платформы XOne зависимость от оценки критиков появляется только для оценок больше 60, с ростом оценки увеличивается объем продаж
* Оценка пользователей не оказывает влияние на продажи

In [ ]:
data_PS4 = data_topical[data_topical['platform'] == 'PS4'].groupby('user_score')['word_sales'].mean()
data_PS4_2 = data_topical[data_topical['platform'] == 'PS4'].groupby('critic_score')['word_sales'].mean()
plt.figure(figsize=(5, 5))
plt.plot(data_PS4_2, 'o-r', alpha=0.9, label="влияние продаж от оценки критиков PS4", 
         lw=5, mec='b', mew=1, ms=5)
plt.plot(data_PS4, 'v-.g', alpha=0.4, label="влияние продаж от оценки пользователей PS4", 
         lw=2, mec='r', mew=2, ms=5)
plt.legend()
plt.grid(True)

* Аналогично с платформой XOne
* Можно отбросить данные по которым рейтинг или не определен или отсутствует, затем помотреть корреляцию

In [ ]:
p_data = data_topical[data_topical['user_score'] > 0] # без пропусков в оценках пользователей
p_data = p_data[p_data['critic_score'] > 0]# без пропусков в оценках пользователей

In [ ]:
p_data['user_score'].corr(p_data['word_sales'])

* Отсутствует зависимость от оценки пользователей количество проданных копий

In [ ]:
p_data['critic_score'].corr(p_data['word_sales'])

* Присутствует слабая зависимость от оценки критиков количество проданных копий

In [ ]:
#Проверим для каждой из платформ зависимость корреляции
platforms = ["PS4", "PS3", "XOne",  "X360", "3DS", "WiiU", 'Wii']
n_platforms = len(platforms)

# список столбцов, по которым хотим построить корреляцию
cols = ["user_score", "critic_score", "word_sales"]

# отрисовываем для каждой платформы
for platform in platforms:
    # отбираем все строки для платформы 
    # и дропаем неинформативные записи среди отобранных
    q = p_data["platform"] == platform
    platform_df = p_data[q].dropna(subset=cols)
    
    # строим корреляции по выбранным столбцам
    correlations = platform_df[cols].corr()
    ax = sns.heatmap(correlations, cmap='rocket_r')
    ax.set_title(platform)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x = p_data['user_score'], y = p_data['word_sales'], 
           color = "blue", edgecolors = "white", linewidths = 0.1, alpha = 0.2)
plt.ylim(0,6)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x = p_data['critic_score'], y = p_data['word_sales'], 
           color = "green", edgecolors = "white", linewidths = 0.1, alpha = 0.2)
plt.ylim(0,6)

### Сформулируем выводы.
* Присутствует слабая зависимость от оценки критиков количество проданных копий для каждой платформы
* Большинство оценок раполагается в диапозоне от 40 до 90
* Отстствует зависимость от оценке пользователей

### Соотнесите выводы с продажами игр на других платформах.
* Выше я рассматривал корреляцию по платформам
* Рассмотрим заисимость продаж от платформы 

In [ ]:
df = data_topical.pivot_table(index="platform", 
                              values=["eu_sales", "jp_sales", "na_sales", 
                                      "other_sales", "word_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(40, 40));

In [ ]:
df = data_topical.pivot_table(index="platform", values=["eu_sales", "word_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(20, 20));


In [ ]:
df = data_topical.pivot_table(index="platform", values=["jp_sales", "other_sales", "na_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(20, 20));

* 1.1 Влияние на мировые продажи по платформам почти полностью соответствуют продажам в Европе. Самые популярные платформы PS4, PS3, X360, XOne, 3DS. Лидирует платформа марки PlayStation Японской компании
* 1.2 Самые популярные платформы в Северной Америке -X360, PS4, PS3, XOne, 3DS. Лидирует платформа марки X-box американской компании
* 1.3 Самые популярные платформы в Японии -3DS, PS3, PSV, PS4, PSP. Платформы Японских компании
* 1.4 Самые популярные платформы в Остальном мире - PS4, PS3, X360, XOne, 3DS.

### Посмотрим на общее распределение игр по жанрам.

In [ ]:
df = data_topical.pivot_table(index="genre", values=[ "jp_sales", "other_sales", "eu_sales", "na_sales", "word_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(50, 50));

In [ ]:
df = data_topical.pivot_table(index="genre", values=["other_sales", "eu_sales", "word_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(20, 20));

In [ ]:
df = data_topical.pivot_table(index="genre", values=["jp_sales", "na_sales"], aggfunc="sum")
df.plot.pie(subplots=True, autopct='%1.1f%%', 
            shadow=True, wedgeprops={'lw':1, 'ls':'--','edgecolor':"k"}, 
            rotatelabels=True, figsize=(20, 20));

### Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

* Как и на продажи по платформам, влияние больше всего оказывает Европа и Остальной мир. Где предпочитают жанр Sport и Action, Shooter. Игры от первого лица. Меньше всего Strategy, Adventure, Puzzle
* В Японии больше всего предпочитают Role-playing, Action, Misc, Fihting, Platform. Меньше всего Shooter, Racing, Strategy
* В Америке Action, Shooter, Sport, Role-playing, Misc. Меньше всего Puzzle, Strategy, Adventure

## Составим портрет пользователя каждого региона
* В предыдущем задании мы составили круговые графики по регионам от жанра, платформы. Обобщим результаты:
* 1.1 Влияние на мировые продажи по платформам почти полностью соответствуют продажам в Европе. Самые популярные платформы PS4, PS3, X360, XOne, 3DS.
* 1.2 Самые популярные платформы в Северной Америке -X360, PS4, PS3, XOne, 3DS.
* 1.3 Самые популярные платформы в Японии -3DS, PS3, PSV, PS4, PSP
* 1.4 Самые популярные платформы в Остальном мире - PS4, PS3, X360, XOne, 3DS.
* 2.1 Самые популярные жанры в Европе и в Остальном мире - Action, Shooter, Sport, Role-Playing, Racing. Игры от первого лица
* 2.2 В Японии больше всего предпочитают Role-playing, Action, Misc, Fihting, Platform. Меньше всего Shooter, Racing
* 2.3 В Америке Action, Shooter, Sport, Role-playing, Misc. Меньше всего Puzzle, Strategy, Adventure
* 3 Рассмотрим влияет ли рейтинг ESRB на продажи в отдельном регионе. 
* 3.1 Entertainment Software Rating Board (ESRB) — негосударственная организация, основное направление деятельности — принятие и определение рейтингов для компьютерных видеоигр и другого развлекательного программного обеспечения в США и Канаде. Больше всего пропусков в рейтинге у игр которые продавались за пределами Севенй Америки, что логично. 

In [ ]:
data_PS4 = data_topical[data_topical['platform'] == 'PS4'].groupby('rating')['na_sales'].mean()
data_PS3 = data_topical[data_topical['platform'] == 'PS3'].groupby('rating')['na_sales'].mean()
data_XOne = data_topical[data_topical['platform'] == 'XOne'].groupby('rating')['na_sales'].mean()
data_3DS = data_topical[data_topical['platform'] == '3DS'].groupby('rating')['na_sales'].mean()
plt.figure(figsize=(5, 5))
plt.plot(data_PS4, 'v-.g', alpha=0.4, label="влияние продаж от оценки ESRB для PS4 в Северной Америке", 
         lw=2, mec='r', mew=2, ms=5)
plt.plot(data_PS3, 'v-.m', alpha=0.4, label="влияние продаж от оценки ESRB для PS3 в Северной Америке", 
         lw=2, mec='r', mew=2, ms=5)
plt.plot(data_XOne, 'v-.c', alpha=0.4, label="влияние продаж от оценки ESRB для XOne в Северной Америке", 
         lw=2, mec='r', mew=2, ms=5)
plt.plot(data_3DS, 'v-.g', alpha=0.4, label="влияние продаж от оценки ESRB для 3DS в Северной Америке", 
         lw=2, mec='r', mew=2, ms=5)
plt.legend()
plt.grid(True)



* 3.2 Рейтинг ESRB влияет на продажи игр. Больше всего игр покупают для взрослых с рейтингом "М". Затем идет для всех "Е". Потом для старше 10 "Е10+". Только потом для подростков "Т". Исключение платформа 3 DS у нее мало игр для взрослых. Анализируем только данные по Северной Америке.

In [ ]:
data_topical #актуальный период
data_topical.pivot_table(index=["rating", "year_of_release"], 
columns="platform", values="word_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True,);
# Количество продаж от оценки по годам за актуальный период в Мире

In [ ]:
data_topical.pivot_table(index=["rating", "year_of_release"], 
columns="platform", values="na_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
# Количество продаж от оценки по годам за актуальный период в Северной Америке


In [ ]:
data_topical.pivot_table(index=["rating", "year_of_release"], 
columns="platform", values="other_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
# Количество продаж от оценки по годам за актуальный период в Остальном мире

In [ ]:
data_topical.pivot_table(index=["rating", "year_of_release"], 
columns="platform", values="jp_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
# Количество продаж от оценки по годам за актуальный период в Японии

In [ ]:
data_topical.pivot_table(index=["rating", "year_of_release"], 
columns="platform", values="eu_sales", aggfunc="sum").plot.bar(stacked=True, figsize=(10, 10), grid=True);
# Количество продаж от оценки по годам за актуальный период в Европе

* 3.2 Рейтинг ESRB влияет на продажи игр. Больше всего игр покупают для взрослых с рейтингом "М". Затем идет для всех "Е". Потом для старше 10 "Е10+". Только потом для подростков "Т". Для Северной Америке 
* ДЛя остального мира, европы похожая тенденци. Но на результат может оказать влияния отсутствия рейтинга у многих игр.
* не рассматриваем данные по Японии, т.к. количество игр без оценок ESRB там слишком велико. Что может исказить реультаты.


<div class="alert alert-info"> <b>Комментарий студента:</b> Определили влияние рейтинга ESRB из разных регионах в актуальном периоде.</div>

## Проверим гипотезы

In [ ]:
from scipy import stats as st

* H_0: Средние пользовательские рейтинги платформ Xbox One = Средние пользовательские рейтинги платформ PC
* H_a: Средние пользовательские рейтинги платформ Xbox One ≠ Средние пользовательские рейтинги платформ PC
* alpha = 0.05

In [ ]:
alpha = .05
#создадим соответствующие переменные
platform_XOne = p_data[(p_data['platform'] == 'XOne')]['user_score']
platform_PC = p_data[(p_data['platform'] == 'PC')]['user_score']
#проводим тест
results = st.ttest_ind(platform_XOne.dropna(), platform_PC.dropna(), equal_var=False)
print(results.pvalue)# вывод значения p-value на экран 
if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")# условный оператор с выводом строки с ответом

* H_0: Средние пользовательские рейтинги жанров Action = Средние пользовательские рейтинги жанров Sports
* H_a: Средние пользовательские рейтинги жанров Action ≠ Средние пользовательские рейтинги жанров Sports
* alpha = 0.05

In [ ]:
alpha = .05
#создадим соответствующие переменные
genre_action = p_data[(p_data['genre'] == 'Action')]['user_score']
genre_sport = p_data[(p_data['genre'] == 'Sports')]['user_score']
#проводим тест
results = st.ttest_ind(genre_action, genre_sport, equal_var=False)
print(results.pvalue)# вывод значения p-value на экран 
if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")# условный оператор с выводом строки с ответом

### Какой критерий применили для проверки гипотез и почему.
Чтобы проверить гипотезу о равенстве среднего двух генеральных совокупностей по взятым из них выборкам, применили метод scipy.stats.ttest_ind()

## Общий вывод
В цели задачи мы ставили выявить определяющие успешность игры закономерности. Что позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
* 1.1 Влияние на мировые продажи по платформам почти полностью соответствуют продажам в Европе. Самые популярные платформы PS4, PS3, X360, XOne, 3DS. Лидирует платформа марки PlayStation Японской компании
* 1.2 Самые популярные платформы в Северной Америке -X360, PS4, PS3, XOne, 3DS. Лидирует платформа марки X-box американской компании
* 1.3 Самые популярные платформы в Японии -3DS, PS3, PSV, PS4, PSP. Платформы Японских компании
* 1.4 Самые популярные платформы в Остальном мире - PS4, PS3, X360, XOne, 3DS.

* 2.1 C хорошей оценкой критиков выше 80. 
* 2.2 Европа и Остальной мир предпочитают жанр Sport и Action, Shooter.
* 2.3 В Японии больше всего предпочитают Role-playing, Action, Misc, Fihting, Platform. Меньше всего Shooter, Racing, Strategy
* 2.4 В Америке Action, Shooter, Sport, Role-playing, Misc. 

* С рейтингом ESRB для всех "М", кроме Японии.
* Также проверили две гипотезы:
1. Средние пользовательские рейтинги платформ Xbox One = Средние пользовательские рейтинги платформ PC (Не получилось отвергнуть гипотезу)
2. Средние пользовательские рейтинги жанров Action = Средние пользовательские рейтинги жанров Sports (Отвергаем нулевую гипотезу)

## Количество продаж в мире по годам и платформам bar_chart_race

In [ ]:
df = data.pivot_table(index="year_of_release", columns="platform", 
                 values="word_sales", aggfunc="sum")

df.fillna(0, inplace=True)
df.sort_values(list(df.columns),inplace=True)
df = df.sort_index()
df.iloc[:, 0:-1] = df.iloc[:, 0:-1].cumsum()
best_platforms = set()

for index, row in df.iterrows():
    best_platforms |= set(row[row > 0].sort_values(ascending=False).head(6).index)

bcr.bar_chart_race(df = df, 
                   n_bars = 6, 
                   sort='desc',
                   title='Best Platforms by Year',
                   fixed_max=True,
                   steps_per_period=20,
                   period_length=1500,
                   filter_column_colors=True);